In [1]:
import numpy as np
import bosonic_qiskit
import qiskit
from sklearn.datasets import make_classification
import pandas as pd
import scipy as sp

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Sklearn Libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA

# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    mean_squared_error, mean_absolute_error, r2_score, roc_curve
)

# System and Warning Handling
import os
import warnings


# ==========================================
# 1. DATA GENERATION (Simulated Fraud Data)
# ==========================================
def get_data():
    # Reduced features to 4 for speed (Manual Gradients are slow!)
    X, y = make_classification(n_samples=300, n_features=4, n_informative=4, 
                               n_redundant=0, weights=[0.9, 0.1], random_state=42)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

X_train, X_test, y_train, y_test = get_data()

# ==========================================
# 2. HYBRID MODEL (Forward Pass)
# ==========================================
def apply_kerr(circuit, kerr_param, qumode, cutoff):
    """Manual Kerr Gate using SNAP"""
    for n in range(cutoff):
        phase = kerr_param * (n ** 2)
        if abs(phase) > 1e-4:
            circuit.cv_snap(phase, n, qumode, None)

def get_probs(result):
    """Extract probabilities for Class 0 (|1,0>) and Class 1 (|0,1>)"""
    counts = result.get_counts()
    total_shots = sum(counts.values())
    p_10, p_01 = 0, 0
    
    for bitstr, count in counts.items():
        val = int(bitstr.replace(" ", ""), 2)
        # Assuming 2 qubits per mode (Cutoff=4)
        n_a = val & 3
        n_b = (val >> 2) & 3
        
        if n_a == 1 and n_b == 0: p_10 += count
        elif n_a == 0 and n_b == 1: p_01 += count
            
    return (p_10/total_shots, p_01/total_shots) if total_shots > 0 else (0,0)

def hybrid_forward(params, x_input):
    # --- Classical NN (4 inputs -> 8 outputs) ---
    # Reduced size for speed
    n_in, n_out = 4, 8
    n_classical = n_in * n_out + n_out
    
    w = params[:n_in*n_out].reshape(n_in, n_out)
    b = params[n_in*n_out : n_classical]
    
    # NN Output used as parameters for the encoding layer
    nn_out = np.tanh(np.dot(x_input, w) + b)
    
    # --- Quantum Circuit ---
    qmr = bosonic_qiskit.QumodeRegister(num_qumodes=2, num_qubits_per_qumode=2)
    cr = qiskit.ClassicalRegister(4)
    circuit = bosonic_qiskit.CVCircuit(qmr, cr)
    
    # Encoding (Driven by NN output)
    # Using first 8 params for: Sq(0), Sq(1), BS, R(0), R(1), D(0), D(1), Kerr(0)
    circuit.cv_sq(nn_out[0], qmr[0])
    circuit.cv_sq(nn_out[1], qmr[1])
    circuit.cv_bs(nn_out[2], qmr[0], qmr[1])
    circuit.cv_r(nn_out[3], qmr[0])
    circuit.cv_d(nn_out[4], qmr[0])
    apply_kerr(circuit, nn_out[5], qmr[0], 4)
    
    # Variational Layer (Trainable Quantum Params)
    q_params = params[n_classical:]
    circuit.cv_bs(q_params[0], qmr[0], qmr[1])
    circuit.cv_r(q_params[1], qmr[0])
    circuit.cv_snap(q_params[2], 1, qmr[0], None) # Non-linearity

    circuit.cv_measure(qmr, cr)
    
    # Simulate
    try:
        _, res, *_ = bosonic_qiskit.util.simulate(circuit, shots=100)
        return get_probs(res)
    except:
        return 0.5, 0.5

# ==========================================
# 3. MANUAL GRADIENT CALCULATION
# ==========================================
def get_batch_loss(params, X_batch, y_batch):
    loss = 0
    scores = []
    
    for x, y in zip(X_batch, y_batch):
        p_10, p_01 = hybrid_forward(params, x)
        
        # Loss: MSE distance from ideal probability
        # Class 0 -> Target p_10=1.0 (p_01=0.0)
        # Class 1 -> Target p_01=1.0 
        target = 1.0 if y == 1 else 0.0
        pred = p_01
        
        loss += (pred - target) ** 2
        scores.append(pred) # Score for AUC is p_01 (Fraud Probability)
        
    return loss / len(y_batch), scores

def compute_gradients_finite_diff(params, X_batch, y_batch, epsilon=0.05):
    """
    Manually calculates gradient for EACH parameter.
    Run time = 2 * N_params * Batch_Size * Simulator_Time
    """
    grads = np.zeros_like(params)
    
    # Base loss (optional, for debugging)
    # base_loss, _ = get_batch_loss(params, X_batch, y_batch)
    
    print(f"  Computing Gradients for {len(params)} parameters...", end="", flush=True)
    
    for i in range(len(params)):
        # Shift Right
        params[i] += epsilon
        loss_plus, _ = get_batch_loss(params, X_batch, y_batch)
        
        # Shift Left
        params[i] -= 2 * epsilon # Move back to -epsilon
        loss_minus, _ = get_batch_loss(params, X_batch, y_batch)
        
        # Reset
        params[i] += epsilon 
        
        # Central Difference
        grads[i] = (loss_plus - loss_minus) / (2 * epsilon)
        
        if i % 10 == 0: print(".", end="", flush=True)
            
    print(" Done.")
    return grads

# ==========================================
# 4. TRAINING LOOP (MAXIMIZE AUC)
# ==========================================
# Setup Parameters: 4*8 (Weights) + 8 (Bias) + 3 (Quantum) = 43 params
n_params = (4 * 8) + 8 + 3
weights = np.random.normal(0, 0.1, n_params)

# Hyperparameters
learning_rate = 0.05
epochs = 5 # Small number due to simulation time
batch_size = 4 

best_val_auc = 0
best_weights = None

print(f"Starting Training with Manual Gradients on {n_params} parameters.")

for epoch in range(epochs):
    # 1. Mini-Batch Training
    indices = np.random.choice(len(X_train), batch_size, replace=False)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    
    # 2. Compute Manual Gradients
    grads = compute_gradients_finite_diff(weights, X_batch, y_batch)
    
    # 3. Update Weights (Gradient Descent)
    weights = weights - (learning_rate * grads)
    
    # 4. Validation & AUC Check
    # We evaluate on a hold-out set to find the Best AUC model
    val_preds = []
    val_loss = 0
    val_indices = np.random.choice(len(X_test), 10, replace=False) # Subset for speed
    
    for idx in val_indices:
        p_10, p_01 = hybrid_forward(weights, X_test[idx])
        val_preds.append(p_01)
        val_loss += (p_01 - (1 if y_test[idx]==1 else 0))**2
        
    try:
        # Calculate AUC (needs both classes present in the batch)
        if len(set(y_test[val_indices])) > 1:
            current_auc = roc_auc_score(y_test[val_indices], val_preds)
        else:
            current_auc = 0.5
            
        print(f"Epoch {epoch+1}: Loss={val_loss/10:.3f} | AUC={current_auc:.3f}")
        
        # SAVE THE MODEL WITH BEST AUC
        if current_auc > best_val_auc:
            best_val_auc = current_auc
            best_weights = weights.copy()
            print("  >>> New Best AUC! Model Saved.")
            
    except Exception as e:
        print(f"  Metric Calc Error: {e}")

# ==========================================
# 5. FINAL TEST
# ==========================================
print("\nFinal Evaluation on Test Set...")
final_preds = []
for x in X_test[:30]: # Limit for speed
    _, p_01 = hybrid_forward(best_weights if best_weights is not None else weights, x)
    final_preds.append(p_01)

final_auc = roc_auc_score(y_test[:30], final_preds)
print(f"Final Test ROC AUC: {final_auc:.4f}")

Starting Training with Manual Gradients on 43 parameters.
  Computing Gradients for 43 parameters........ Done.
Epoch 1: Loss=0.100 | AUC=0.500
  >>> New Best AUC! Model Saved.
  Computing Gradients for 43 parameters........ Done.
Epoch 2: Loss=0.100 | AUC=0.500
  Computing Gradients for 43 parameters........ Done.
Epoch 3: Loss=0.200 | AUC=0.500
  Computing Gradients for 43 parameters........ Done.
Epoch 4: Loss=0.100 | AUC=0.500
  Computing Gradients for 43 parameters........ Done.
Epoch 5: Loss=0.100 | AUC=0.500

Final Evaluation on Test Set...
Final Test ROC AUC: 0.6250


In [ ]:
import numpy as np
import bosonic_qiskit
import qiskit
import random
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score
import cmath

# --- 1. Autograd Engine (Value Class) ---
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __neg__(self):
        return self * Value(-1)

    def log(self):
        val = self.data
        # Clip to prevent log(0) = -inf
        if val < 1e-7: val = 1e-7
        if val > 1.0 - 1e-7: val = 1.0 - 1e-7
        
        out = Value(np.log(val), (self,), 'log')
        def _backward():
            self.grad += (1.0 / val) * out.grad
        out._backward = _backward
        return out
    

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
        
    def tanh(self):
        # Tanh activation allows negative values (Phase Space!)
        x = self.data
        t = (np.exp(2*x) - 1)/(np.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            v._backward()
            
    def zero_grad(self):
        self.grad = 0

# --- 2. Improved Quantum Op (Using a Custom Observable) ---
class QuantumOp(Value):
    def __init__(self, circuit_builder, weights, inputs, cutoff_dim=2):
        self.weights = weights
        self.inputs = inputs
        self.circuit_builder = circuit_builder
        self.cutoff_dim = cutoff_dim
        
        # --- Define the observable for our HYBRID classification task ---
        cv_dim = self.cutoff_dim ** 2
        
        o_cv = np.zeros((cv_dim, cv_dim))
        fraud_idx = 1
        normal_idx = self.cutoff_dim
        o_cv[fraud_idx, fraud_idx] = 1.0
        o_cv[normal_idx, normal_idx] = -1.0
        
        i_qubit = np.identity(2)
        self.observable = np.kron(i_qubit, o_cv)
        
        parents = tuple(weights + inputs)
        
        w_vals = [w.data for w in self.weights]
        i_vals = [i.data for i in self.inputs]
        val = self._run_simulation(w_vals, i_vals)
        super().__init__(val, parents, 'QuantumHybrid')

    def _run_simulation(self, w_vals, i_vals):
        # 1. Setup a HYBRID Circuit
        qmr = bosonic_qiskit.QumodeRegister(num_qumodes=2, num_qubits_per_qumode=self.cutoff_dim)
        qr = qiskit.QuantumRegister(1)
        cr = qiskit.ClassicalRegister(3)
        
        circ = bosonic_qiskit.CVCircuit(qmr, qr, cr)
        
        # 2. Build User Circuit, passing both registers
        self.circuit_builder(circ, w_vals, i_vals, qmr, qr)
        
        try:
            state, _, _ = bosonic_qiskit.util.simulate(circ)
            probs = state.probabilities()
            p_fraud = probs[1]
            return p_fraud
        except Exception as e:
            print(f"Simulation failed: {e}")
            return 0.5

    def _backward(self):
        h = 0.001 
        w_numerics = [w.data for w in self.weights]
        i_numerics = [i.data for i in self.inputs]
        for idx, w in enumerate(self.weights):
            w_copy = w_numerics.copy()
            w_copy[idx] += h
            out_plus = self._run_simulation(w_copy, i_numerics)
            w_copy[idx] -= 2*h
            out_minus = self._run_simulation(w_copy, i_numerics)
            grad = (out_plus - out_minus) / (2*h)
            w.grad += self.grad * grad
        for idx, inp in enumerate(self.inputs):
            i_copy = i_numerics.copy()
            i_copy[idx] += h
            out_plus = self._run_simulation(w_numerics, i_copy)
            i_copy[idx] -= 2*h
            out_minus = self._run_simulation(w_numerics, i_copy)
            grad = (out_plus - out_minus) / (2*h)
            inp.grad += self.grad * grad

# --- 3. Hybrid Model with Tanh & Strong Init ---
class LinearLayer:
    def __init__(self, n_in, n_out):
        self.weights = [[Value(random.gauss(0, 0.2)) for _ in range(n_out)] for _ in range(n_in)]
        self.bias = [Value(0.1) for _ in range(n_out)]
        
    def __call__(self, x):
        out = []
        for j in range(len(self.bias)):
            act = self.bias[j]
            for i in range(len(x)):
                act = act + x[i] * self.weights[i][j]
            out.append(act)
        return out
    
    def parameters(self):
        return [p for row in self.weights for p in row] + self.bias

class HybridFraudDetector:
    def __init__(self, n_features):
        self.c_layer1 = LinearLayer(n_features, 20)
        self.c_layer2 = LinearLayer(20, 14) 
        
        self.q_weights = []
        for i in range(21):
            val = random.uniform(-0.1, 0.1) 
            self.q_weights.append(Value(val))
            
    def forward(self, x_raw):
        x_vals = [Value(xi) for xi in x_raw]
        h1 = self.c_layer1(x_vals)
        h1_act = [h.tanh() for h in h1]
        encoding_params = self.c_layer2(h1_act) 
        out = QuantumOp(self.quantum_circuit, self.q_weights, encoding_params)
        return out
    
    @staticmethod
    def quantum_circuit(circuit, weights, inputs, qmr, qr):
        def to_complex(mag, phase):
            return mag * cmath.exp(1j * phase)

        sq_0 = to_complex(inputs[0], inputs[1])
        sq_1 = to_complex(inputs[2], inputs[3])
        circuit.cv_sq(sq_0, qmr[0])
        circuit.cv_sq(sq_1, qmr[1])
        circuit.cv_bs(inputs[4], qmr[0], qmr[1])
        circuit.cv_r(inputs[6], qmr[0])
        circuit.cv_r(inputs[7], qmr[1])
        d_0 = to_complex(inputs[8], inputs[9])
        d_1 = to_complex(inputs[10], inputs[11])
        circuit.cv_d(d_0, qmr[0])
        circuit.cv_d(d_1, qmr[1])
        
        circuit.cv_bs(weights[0], qmr[0], qmr[1])
        circuit.cv_r(weights[2], qmr[0])
        circuit.cv_r(weights[3], qmr[1])
        s_var_0 = to_complex(weights[4], weights[5])
        s_var_1 = to_complex(weights[6], weights[7])
        circuit.cv_sq(s_var_0, qmr[0])
        circuit.cv_sq(s_var_1, qmr[1])
        circuit.cv_bs(weights[8], qmr[0], qmr[1])
        circuit.cv_r(weights[10], qmr[0])
        circuit.cv_r(weights[11], qmr[1])
        d_var_0 = to_complex(weights[12], weights[13])
        d_var_1 = to_complex(weights[14], weights[15])
        circuit.cv_d(d_var_0, qmr[0])
        circuit.cv_d(d_var_1, qmr[1])
        
        circuit.h(qr[0])
        circuit.ry(weights[18], qr[0])
        
        cd_alpha = to_complex(weights[19], weights[20])
        circuit.cv_c_d(cd_alpha, [qmr[0]], qr[0])
        
    def parameters(self):
        return self.c_layer1.parameters() + self.c_layer2.parameters() + self.q_weights
    
# --- 4. Training - MODIFIED FOR STABILITY ---
def get_full_data(n_samples=1000):
    try:
        df = pd.read_csv(r"D:\Academic\QML_Intern\Anomaly_Detection\Fraud_Detection\creditcard_data.csv")
    except FileNotFoundError:
        print("ERROR: Please update the path to 'creditcard_data.csv'")
        return None, None, None, None
        
    fraud = df[df['Class'] == 1]
    normal = df[df['Class'] == 0]
    n_per = n_samples // 2
    fraud = resample(fraud, n_samples=n_per, random_state=42)
    normal = resample(normal, n_samples=n_per, random_state=42)
    df_bal = pd.concat([fraud, normal])
    
    X = df_bal.drop(['Class', 'Time'], axis=1).values
    y = df_bal['Class'].values
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Setup
X_train, X_test, y_train, y_test = get_full_data(n_samples=10000)
if X_train is not None:
    model = HybridFraudDetector(n_features=29)
    optimizer_params = model.parameters()
    epochs = 10
    
    # --- CHANGE 1: Lower the learning rate ---
    # Cross-entropy can produce large gradients, so a smaller LR is needed for stability.
    lr = 0.001 
    
    print(f"Training with BINARY CROSS-ENTROPY loss...")
    
    for epoch in range(epochs):
        epoch_loss = 0
        start = time.time()
        indices = np.random.permutation(len(X_train))
        
        for i in range(0, len(X_train), 64):
            batch_idx = indices[i:i+64]
            batch_loss = Value(0)
            
            for idx in batch_idx:
                # pred is now a probability p in [0, 1]
                p = model.forward(X_train[idx])
                y_target = y_train[idx]
                
                # --- CHANGE FOR CROSS-ENTROPY ---
                # Loss = -(y*log(p) + (1-y)*log(1-p))
                y = Value(y_target)
                term = (y * p.log() + (Value(1) + y.__neg__()) * (Value(1) + p.__neg__()).log()).__neg__()
                
                batch_loss = batch_loss + term
                
            batch_loss = batch_loss * Value(1.0/64)
            
            # Zero Grads & Backprop
            for p in optimizer_params: p.zero_grad()
            batch_loss.backward()
            
            # --- CHANGE 2: Implement Gradient Clipping ---
            # This prevents exploding gradients from destabilizing the training.
            clip_value = 1.0 # Do not allow any single gradient to be larger than this.
            for p in optimizer_params:
                p.grad = np.clip(p.grad, -clip_value, clip_value)

            # Update weights
            for p in optimizer_params:
                p.data -= lr * p.grad
                
            epoch_loss += batch_loss.data
            print(f"\rBatch {i//64} | Loss: {batch_loss.data:.4f}", end="")
            
        print(f"\nEpoch {epoch+1} | Avg Loss: {epoch_loss / (len(X_train)/64):.4f} | Time: {time.time()-start:.1f}s")
        
        # Test AUC (still works with probability outputs)
        preds = [model.forward(x).data for x in X_test]
        try:
            print(f"Test AUC: {roc_auc_score(y_test, preds):.4f}")
        except: pass


Training with MEAN PHOTON metric...


d:\Academic\Dual Degree Project\DDP\boson_env\lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csc_array is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


Batch 124 | Loss: 1.0671
Epoch 1 | Avg Loss: 1.0822 | Time: 839.5s
Test AUC: 0.3764
Batch 1 | Loss: 0.9781

d:\Academic\Dual Degree Project\DDP\boson_env\lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csc_array is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


Batch 4 | Loss: 1.0767

KeyboardInterrupt: 